In [174]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams
from nba_api.stats.endpoints import boxscoretraditionalv3
import pandas as pd
import datetime
import time

In [175]:
import pandas as pd

# get all NBA teams
nba_teams = teams.get_teams()
teams_df = pd.DataFrame(nba_teams)

# save team IDs and names
team_ids = teams_df['id'].tolist()

teams_df.head()


,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [176]:
# get current season
current_year = datetime.datetime.now().year
current_month = datetime.datetime.now().month
season_id = f"2{current_year - 1}" if current_month < 10 else f"2{current_year}"

# get schedules for all seasons
all_games = []

for team_id in team_ids:
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
    games = gamefinder.get_data_frames()[0]
    games_current_season = games[games['SEASON_ID'] == season_id]  # Filter for current season
    all_games.append(games_current_season)

# collect team schedules into single dataframe
games_df = pd.concat(all_games, ignore_index=True)

# save data and view
games_df.to_csv(f'games_{season_id}.csv', index=False)
games_df.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612737,ATL,Atlanta Hawks,0022400701,2025-02-03,ATL @ DET,W,240,132,...,0.821,5.0,32.0,37.0,27,8.0,7,12,27,2.0
1,22024,1610612737,ATL,Atlanta Hawks,0022400686,2025-02-01,ATL @ IND,L,240,127,...,0.719,11.0,33.0,44.0,31,6.0,4,12,23,-5.0
2,22024,1610612737,ATL,Atlanta Hawks,0022400675,2025-01-30,ATL @ CLE,L,240,115,...,0.741,10.0,26.0,36.0,28,10.0,1,11,18,-22.0
3,22024,1610612737,ATL,Atlanta Hawks,0022400532,2025-01-28,ATL vs. HOU,L,240,96,...,0.813,9.0,31.0,40.0,19,11.0,5,16,18,-4.0
4,22024,1610612737,ATL,Atlanta Hawks,0022400656,2025-01-27,ATL @ MIN,L,241,92,...,0.667,14.0,37.0,51.0,24,10.0,3,21,20,-8.0


In [177]:
# get player stats
all_player_stats = []

for index, row in games_df.iterrows():
    print(f'on game {row["GAME_ID"]}')
    game_id = row['GAME_ID']
    try:
        # fetch box score for the game
        boxscore = boxscoretraditionalv3.BoxScoreTraditionalV3(game_id=game_id)
        player_stats = boxscore.player_stats.get_data_frame()

        # add game context to player stats
        player_stats['GAME_ID'] = game_id
        player_stats['GAME_DATE'] = row['GAME_DATE']
        player_stats['TEAM_NAME'] = row['TEAM_NAME']
        all_player_stats.append(player_stats)

        # respect API rate limits
        time.sleep(1)

    except Exception as e:
        print(f"Error fetching player stats for game {game_id}: {e}")

# collect player stats into single dataframe
player_stats_df = pd.concat(all_player_stats, ignore_index=True)

# save data and view
player_stats_df.to_csv(f'player_stats_{season_id}.csv', index=False)
player_stats_df.head()

on game 0022400701
on game 0022400686
on game 0022400675
on game 0022400532
on game 0022400656
on game 0022400639
on game 0022400623
on game 0022400612
on game 0022400602
on game 0022400587
on game 0022400563
on game 0022400556
on game 0022400522
on game 0022400506
on game 0022400486
on game 0022400477
on game 0022400461
on game 0022400438
on game 0022400427
on game 0022400413
on game 0022400395
on game 0022400378
on game 0022400370
on game 0022401229
on game 0022401202
on game 0022400350
on game 0022400334
on game 0022400323
on game 0022400315
on game 0022400300
on game 0022400041
on game 0022400287
on game 0022400280
on game 0022400030
on game 0022400258
on game 0022400250
on game 0022400239
on game 0022400012
on game 0022400001
on game 0022400198
on game 0022400185
on game 0022400171
on game 0022400157
on game 0022400152
on game 0022400135
on game 0022400121
on game 0022400103
on game 0022400100
on game 0022400079
on game 0022400064
on game 1522400064
on game 1522400055
on game 1522

KeyboardInterrupt: 

In [ ]:
duplicates = player_stats_df.duplicated(subset=['gameId', 'personId'], keep=False)
player_stats_cleaned = player_stats_df.drop_duplicates(subset=['gameId', 'personId'], keep='first')
print(f'player_stats_df: {len(player_stats_df)}         player_stats_cleaned: {len(player_stats_cleaned)}')

player_stats_df: 15952         player_stats_cleaned: 12470


In [ ]:
player_stats_cleaned.head()

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,...,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints,GAME_ID,GAME_DATE,TEAM_NAME
0,0022400563,1610612737,Atlanta,Hawks,ATL,hawks,1630249,Vít,Krejčí,V. Krejčí,...,6,0,2,1,1,11,21.0,0022400563,2025-01-15,Atlanta Hawks
1,0022400563,1610612737,Atlanta,Hawks,ATL,hawks,1631223,David,Roddy,D. Roddy,...,2,3,0,1,0,4,7.0,0022400563,2025-01-15,Atlanta Hawks
2,0022400563,1610612737,Atlanta,Hawks,ATL,hawks,203991,Clint,Capela,C. Capela,...,1,0,1,0,2,11,8.0,0022400563,2025-01-15,Atlanta Hawks
3,0022400563,1610612737,Atlanta,Hawks,ATL,hawks,1630700,Dyson,Daniels,D. Daniels,...,3,3,1,2,1,18,9.0,0022400563,2025-01-15,Atlanta Hawks
4,0022400563,1610612737,Atlanta,Hawks,ATL,hawks,1630811,Keaton,Wallace,K. Wallace,...,6,4,0,1,3,27,29.0,0022400563,2025-01-15,Atlanta Hawks


In [ ]:
# merge datasets on GAME_ID
merged_df = pd.merge(player_stats_df, games_df, 
                     left_on=['gameId', 'teamId'],
                     right_on=['GAME_ID','TEAM_ID'], 
                     suffixes=('', '_game'), 
                     how='left')

# drop duplicate columns
merged_df.drop(columns=['gameId', 'teamId'], inplace=True)

# save and view
merged_df.to_csv(f'merged_stats_{season_id}.csv', index=False)
merged_df.head(8)



,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,playerSlug,position,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,Atlanta,Hawks,ATL,hawks,1630249,Vít,Krejčí,V. Krejčí,vít-krejčí,F,...,0.682,14.0,37.0,51.0,28.0,11.0,5.0,9.0,15.0,16.0
1,Atlanta,Hawks,ATL,hawks,1631223,David,Roddy,D. Roddy,david-roddy,F,...,0.682,14.0,37.0,51.0,28.0,11.0,5.0,9.0,15.0,16.0
2,Atlanta,Hawks,ATL,hawks,203991,Clint,Capela,C. Capela,clint-capela,C,...,0.682,14.0,37.0,51.0,28.0,11.0,5.0,9.0,15.0,16.0
3,Atlanta,Hawks,ATL,hawks,1630700,Dyson,Daniels,D. Daniels,dyson-daniels,G,...,0.682,14.0,37.0,51.0,28.0,11.0,5.0,9.0,15.0,16.0
4,Atlanta,Hawks,ATL,hawks,1630811,Keaton,Wallace,K. Wallace,keaton-wallace,G,...,0.682,14.0,37.0,51.0,28.0,11.0,5.0,9.0,15.0,16.0
5,Atlanta,Hawks,ATL,hawks,1630168,Onyeka,Okongwu,O. Okongwu,onyeka-okongwu,,...,0.682,14.0,37.0,51.0,28.0,11.0,5.0,9.0,15.0,16.0
6,Atlanta,Hawks,ATL,hawks,203992,Bogdan,Bogdanović,B. Bogdanović,bogdan-bogdanović,,...,0.682,14.0,37.0,51.0,28.0,11.0,5.0,9.0,15.0,16.0
7,Atlanta,Hawks,ATL,hawks,1629726,Garrison,Mathews,G. Mathews,garrison-mathews,,...,0.682,14.0,37.0,51.0,28.0,11.0,5.0,9.0,15.0,16.0


In [ ]:
merged_df.columns

Index(['teamCity', 'teamName', 'teamTricode', 'teamSlug', 'personId',
       'firstName', 'familyName', 'nameI', 'playerSlug', 'position', 'comment',
       'jerseyNum', 'minutes', 'fieldGoalsMade', 'fieldGoalsAttempted',
       'fieldGoalsPercentage', 'threePointersMade', 'threePointersAttempted',
       'threePointersPercentage', 'freeThrowsMade', 'freeThrowsAttempted',
       'freeThrowsPercentage', 'reboundsOffensive', 'reboundsDefensive',
       'reboundsTotal', 'assists', 'steals', 'blocks', 'turnovers',
       'foulsPersonal', 'points', 'plusMinusPoints', 'GAME_ID', 'GAME_DATE',
       'TEAM_NAME', 'SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'TEAM_NAME_game', 'GAME_ID_game', 'GAME_DATE_game', 'MATCHUP', 'WL',
       'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PLUS_MINUS'],
      dtype='object')

## Data Preprocessing

In [ ]:
# get rolling averages for players for each game
# sort data by game date
merged_df = merged_df.sort_values(by=['personId', 'GAME_DATE'])

# get list of stats to average
roll_list = ['minutes', 'fieldGoalsMade', 'fieldGoalsAttempted',
       'fieldGoalsPercentage', 'threePointersMade', 'threePointersAttempted',
       'threePointersPercentage', 'freeThrowsMade', 'freeThrowsAttempted',
       'freeThrowsPercentage', 'reboundsOffensive', 'reboundsDefensive',
       'reboundsTotal', 'assists', 'steals', 'blocks', 'turnovers',
       'foulsPersonal', 'points', 'plusMinusPoints']

# change minutes column to numeric
# merged_df['MIN'] = (
#     merged_df['MIN']
#     .str.split(':').str[0] 
#     .astype(float)           
# )

# generate player rolling stats
for stat in roll_list:
    merged_df[f'ROLLING_{stat}'] = merged_df.groupby('personId')[stat].rolling(5, min_periods=1).mean().reset_index(level=0, drop=True)

merged_df.head(6)

In [ ]:
len(merged_df)

16066

In [ ]:
# get detailed shot chart data and clutch data
from nba_api.stats.endpoints import shotchartdetail, commonteamyears, commonallplayers

players = commonallplayers.CommonAllPlayers(is_only_current_season=1).get_data_frames()[0]
player_team_map = players[['PERSON_ID', 'TEAM_ID']].drop_duplicates()

shot_chart_data = []
clutch_data = []

for _, row in player_team_map.iterrows():
    try:
        player_id = row['PERSON_ID']
        team_id = row['TEAM_ID']

        scd = shotchartdetail.ShotChartDetail(team_id=team_id,
                                            player_id=player_id,
                                            season_type_all_star='Regular Season',
                                            context_measure_simple='FGA')
        scd_data = scd.get_data_frames()[0]

        player_overalls = scd_data.groupby('PLAYER_ID').agg(
            total_shots=('SHOT_ATTEMPTED_FLAG', 'sum'),
            total_makes=('SHOT_MADE_FLAG', 'sum'),
            fg_pct=('SHOT_MADE_FLAG', lambda x: x.sum() / len(x) if len(x) > 0 else 0),
            paint_shots=('SHOT_ZONE_BASIC', lambda x: (x == 'Restricted Area').sum()),
            mid_range_shots=('SHOT_ZONE_BASIC', lambda x: (x == 'Mid-Range').sum()),
            three_point_shots=('SHOT_ZONE_BASIC', lambda x: (x == 'Above the Break 3').sum()),
            avg_distance=('SHOT_DISTANCE', 'mean'),
        ).reset_index()

        shot_chart_data.append(player_overalls)

        scd_c = shotchartdetail.ShotChartDetail(team_id=team_id,
                                            player_id=player_id,
                                            season_type_all_star='Regular Season',
                                            context_measure_simple='FGA',
                                            clutch_time_nullable='Last 5 Minutes')
        
        scd_c_data = scd_c.get_data_frames()[0]
        
        player_overalls_c = scd_c_data.groupby('PLAYER_ID').agg(
            total_shots_clutch=('SHOT_ATTEMPTED_FLAG', 'sum'),
            total_makes_clutch=('SHOT_MADE_FLAG', 'sum'),
            fg_pct_clutch=('SHOT_MADE_FLAG', lambda x: x.sum() / len(x) if len(x) > 0 else 0),
            paint_shots_clutch=('SHOT_ZONE_BASIC', lambda x: (x == 'Restricted Area').sum()),
            mid_range_shots_clutch=('SHOT_ZONE_BASIC', lambda x: (x == 'Mid-Range').sum()),
            three_point_shots_clutch=('SHOT_ZONE_BASIC', lambda x: (x == 'Above the Break 3').sum()),
            avg_distance_clutch=('SHOT_DISTANCE', 'mean'),
        ).reset_index()

        clutch_data.append(player_overalls_c)
    except Exception as e:
        print(f'{e}: error fetching player {player_id}')

shot_chart_data = pd.concat(shot_chart_data, ignore_index=True)
clutch_data = pd.concat(clutch_data, ignore_index=True)

shot_data = pd.merge(shot_chart_data, clutch_data, on='PLAYER_ID', how='left')

shot_data.head()

HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30): error fetching player 1642271
HTTPSConnectionPool(host='stats.nba.com', port=443): Max retries exceeded with url: /stats/shotchartdetail?AheadBehind=&ClutchTime=&ContextFilter=&ContextMeasure=FGA&DateFrom=&DateTo=&EndPeriod=&EndRange=&GameID=&GameSegment=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&Period=0&PlayerID=1631204&PlayerPosition=&PointDiff=&Position=&RangeType=&RookieYear=&Season=&SeasonSegment=&SeasonType=Regular+Season&StartPeriod=&StartRange=&TeamID=1610612765&VsConference=&VsDivision= (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x16ad67b60>, 'Connection to stats.nba.com timed out. (connect timeout=30)')): error fetching player 1631204


/var/folders/vl/05t5ksb558l04dj_v01x4f4r0000gn/T/ipykernel_47668/845230661.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  shot_chart_data = pd.concat(shot_chart_data, ignore_index=True)
/var/folders/vl/05t5ksb558l04dj_v01x4f4r0000gn/T/ipykernel_47668/845230661.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  clutch_data = pd.concat(clutch_data, ignore_index=True)


,PLAYER_ID,total_shots,total_makes,fg_pct,paint_shots,mid_range_shots,three_point_shots,avg_distance,total_shots_clutch,total_makes_clutch,fg_pct_clutch,paint_shots_clutch,mid_range_shots_clutch,three_point_shots_clutch,avg_distance_clutch
0,1630173,141,76,0.539007,91,5,3,5.347518,7,4,0.571429,5,0,1,7.285714
1,203500,86,50,0.581395,60,0,0,2.953488,3,1,0.333333,3,0,0,1.666667
2,1628389,617,288,0.466775,158,88,84,10.380875,82,47,0.573171,23,13,14,10.902439
3,1630534,350,179,0.511429,153,5,63,12.545714,30,21,0.700000,19,0,3,8.566667
4,1630583,429,211,0.491841,121,20,136,13.993007,29,13,0.448276,13,2,8,10.344828


In [ ]:
shot_chart_data

,PLAYER_ID,total_shots,total_makes,fg_pct,paint_shots,mid_range_shots,three_point_shots,avg_distance
0,1630173,99,51,0.515152,60,2,2,5.717172
1,203500,52,31,0.596154,39,0,0,2.403846
2,1628389,533,242,0.454034,140,67,74,10.294559
3,1630534,329,167,0.507599,146,5,59,12.370821
4,1630583,377,188,0.498674,109,18,118,13.840849
...,...,...,...,...,...,...,...,...
508,1629027,684,275,0.402047,110,71,299,17.067251
509,1627826,447,275,0.615213,243,12,0,3.921700
510,1641783,327,134,0.409786,96,15,95,14.348624
511,1628427,7,4,0.571429,5,0,1,8.285714


In [179]:
merged_df.head()


,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,playerSlug,position,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
13098,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,0.840,15.0,31.0,46.0,22.0,7.0,8.0,7.0,22.0,7.0
13070,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,0.744,6.0,31.0,37.0,33.0,10.0,5.0,12.0,18.0,7.0
13045,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,0.875,13.0,32.0,45.0,32.0,8.0,3.0,19.0,19.0,4.0
13008,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,0.722,11.0,35.0,46.0,25.0,5.0,6.0,11.0,19.0,-4.0
3292,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,0.848,5.0,32.0,37.0,21.0,6.0,0.0,20.0,7.0,-24.0


In [180]:
shot_data[shot_data['PLAYER_ID'] == 2544]

,PLAYER_ID,total_shots,total_makes,fg_pct,paint_shots,mid_range_shots,three_point_shots,avg_distance,total_shots_clutch,total_makes_clutch,fg_pct_clutch,paint_shots_clutch,mid_range_shots_clutch,three_point_shots_clutch,avg_distance_clutch
210,2544,800,410,0.5125,243,145,227,12.92125,83,42,0.506024,26,19,25,14.26506


In [181]:
# rename player id for merged df so it can be merged with shot chart data
merged_df = merged_df.rename(columns={'personId': 'PLAYER_ID'})
merged_df.head()

,teamCity,teamName,teamTricode,teamSlug,PLAYER_ID,firstName,familyName,nameI,playerSlug,position,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
13098,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,0.840,15.0,31.0,46.0,22.0,7.0,8.0,7.0,22.0,7.0
13070,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,0.744,6.0,31.0,37.0,33.0,10.0,5.0,12.0,18.0,7.0
13045,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,0.875,13.0,32.0,45.0,32.0,8.0,3.0,19.0,19.0,4.0
13008,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,0.722,11.0,35.0,46.0,25.0,5.0,6.0,11.0,19.0,-4.0
3292,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,0.848,5.0,32.0,37.0,21.0,6.0,0.0,20.0,7.0,-24.0


In [182]:
# perform merge
merged_df = pd.merge(merged_df, shot_data, on='PLAYER_ID', how='left')
merged_df.fillna(0, inplace=True)

merged_df.head()

/var/folders/vl/05t5ksb558l04dj_v01x4f4r0000gn/T/ipykernel_47668/3129978357.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df.fillna(0, inplace=True)


,teamCity,teamName,teamTricode,teamSlug,PLAYER_ID,firstName,familyName,nameI,playerSlug,position,...,mid_range_shots,three_point_shots,avg_distance,total_shots_clutch,total_makes_clutch,fg_pct_clutch,paint_shots_clutch,mid_range_shots_clutch,three_point_shots_clutch,avg_distance_clutch
0,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,145,227,12.92125,83,42,0.506024,26,19,25,14.26506
1,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,145,227,12.92125,83,42,0.506024,26,19,25,14.26506
2,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,145,227,12.92125,83,42,0.506024,26,19,25,14.26506
3,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,145,227,12.92125,83,42,0.506024,26,19,25,14.26506
4,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,145,227,12.92125,83,42,0.506024,26,19,25,14.26506


In [ ]:
# get advanced metrics


In [186]:
# get rolling averages for players for each game
# sort data by game date
merged_df = merged_df.sort_values(by=['PLAYER_ID', 'GAME_DATE'])

# get list of stats to average
roll_list = ['minutes', 'fieldGoalsMade', 'fieldGoalsAttempted',
       'fieldGoalsPercentage', 'threePointersMade', 'threePointersAttempted',
       'threePointersPercentage', 'freeThrowsMade', 'freeThrowsAttempted',
       'freeThrowsPercentage', 'reboundsOffensive', 'reboundsDefensive',
       'reboundsTotal', 'assists', 'steals', 'blocks', 'turnovers',
       'foulsPersonal', 'points', 'plusMinusPoints']

# change minutes column to numeric
# merged_df['MIN'] = (
#     merged_df['MIN']
#     .str.split(':').str[0] 
#     .astype(float)           
# )

# generate player rolling stats
for stat in roll_list:
    merged_df[f'ROLLING_{stat}'] = merged_df.groupby('PLAYER_ID')[stat].rolling(5, min_periods=1).mean().reset_index(level=0, drop=True)

merged_df.head(6)

DataError: No numeric types to aggregate

In [191]:
numeric_columns = merged_df.select_dtypes(include=['number']).columns
numeric_columns

Index(['PLAYER_ID', 'fieldGoalsMade', 'fieldGoalsAttempted',
       'fieldGoalsPercentage', 'threePointersMade', 'threePointersAttempted',
       'threePointersPercentage', 'freeThrowsMade', 'freeThrowsAttempted',
       'freeThrowsPercentage', 'reboundsOffensive', 'reboundsDefensive',
       'reboundsTotal', 'assists', 'steals', 'blocks', 'turnovers',
       'foulsPersonal', 'points', 'plusMinusPoints', 'TEAM_ID', 'MIN', 'PTS',
       'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PLUS_MINUS', 'total_shots', 'total_makes', 'fg_pct', 'paint_shots',
       'mid_range_shots', 'three_point_shots', 'avg_distance',
       'total_shots_clutch', 'total_makes_clutch', 'fg_pct_clutch',
       'paint_shots_clutch', 'mid_range_shots_clutch',
       'three_point_shots_clutch', 'avg_distance_clutch'],
      dtype='object')

In [192]:
merged_df.columns

Index(['teamCity', 'teamName', 'teamTricode', 'teamSlug', 'PLAYER_ID',
       'firstName', 'familyName', 'nameI', 'playerSlug', 'position', 'comment',
       'jerseyNum', 'minutes', 'fieldGoalsMade', 'fieldGoalsAttempted',
       'fieldGoalsPercentage', 'threePointersMade', 'threePointersAttempted',
       'threePointersPercentage', 'freeThrowsMade', 'freeThrowsAttempted',
       'freeThrowsPercentage', 'reboundsOffensive', 'reboundsDefensive',
       'reboundsTotal', 'assists', 'steals', 'blocks', 'turnovers',
       'foulsPersonal', 'points', 'plusMinusPoints', 'GAME_ID', 'GAME_DATE',
       'TEAM_NAME', 'SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'TEAM_NAME_game', 'GAME_ID_game', 'GAME_DATE_game', 'MATCHUP', 'WL',
       'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PLUS_MINUS', 'total_shots', 'total_makes', 'fg_pct',
       'paint_shots', 'mid_range_shots', 'th

In [ ]:
# get advanced metrics

# effective field goal pct
merged_df['efg'] = (merged_df['fieldGoalsMade'] + 0.5 * merged_df['threePointersMade']) / merged_df['fieldGoalsAttempted']

# true shooting pct
merged_df['ts'] = merged_df['points'] / (2 * (merged_df['fieldGoalsAttempted'] + 0.44 * merged_df['freeThrowsAttempted']))

# usage pct
from nba_api.stats.endpoints import boxscoreusagev3
game_ids = merged_df['GAME_ID'].unique()
usage_data = []

for game_id in game_ids:
    try:
        usage = boxscoreusagev3.BoxScoreUsageV3(game_id=game_id)
        usage_df = usage.get_data_frames()[0]
        usage_df = usage_df[['gameId', 'personId', 'usagePercentage']]
        usage_data.append(usage_df)
        time.sleep(0.5)
    except Exception as e:
        print(f"Error fetching game {game_id}: {e}")

usage_data = pd.concat(usage_data, ignore_index=True)
usage_data.rename(columns={'gameId': 'GAME_ID', 'personId': 'PLAYER_ID'}, inplace=True)

In [206]:
# usage pct cont.
merged_df = merged_df.merge(usage_data, on=['GAME_ID', 'PLAYER_ID'], how='left')
merged_df.head()

,teamCity,teamName,teamTricode,teamSlug,PLAYER_ID,firstName,familyName,nameI,playerSlug,position,...,fg_pct_clutch,paint_shots_clutch,mid_range_shots_clutch,three_point_shots_clutch,avg_distance_clutch,efg,ts,win_streak,streak,usagePercentage
0,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,0.506024,26,19,25,14.26506,0.468750,0.486618,1,1,0.214
1,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,0.506024,26,19,25,14.26506,0.615385,0.635593,2,2,0.232
2,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,0.506024,26,19,25,14.26506,0.636364,0.673401,3,3,0.308
3,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,0.506024,26,19,25,14.26506,0.285714,0.330529,-1,-1,0.218
4,Los Angeles,Lakers,LAL,lakers,2544,LeBron,James,L. James,lebron-james,F,...,0.506024,26,19,25,14.26506,0.730769,0.808458,-2,-2,0.310


In [200]:
# win-loss streak
def calculate_streaks(wl_series):
    streak = []
    count = 0
    
    for result in wl_series:
        if result == "W":
            count = count + 1 if count >= 0 else 1
        else:
            count = count - 1 if count <= 0 else -1
        streak.append(count)
    
    return streak

merged_df['streak'] = (
    merged_df.groupby('TEAM_ID')['WL'].transform(calculate_streaks)
)

# 

,TEAM_ID,GAME_DATE,WL,streak
0,1.610613e+09,2024-10-22,W,1
1,1.610613e+09,2024-10-25,W,2
2,1.610613e+09,2024-10-26,W,3
3,1.610613e+09,2024-10-28,L,-1
4,1.610613e+09,2024-10-30,L,-2
...,...,...,...,...
16061,1.610613e+09,2024-12-07,W,2
16062,1.610613e+09,2024-12-23,L,-1
16063,1.610613e+09,2025-01-06,W,1
16064,1.610613e+09,2025-01-09,L,-1
